In [ ]:
from inference.simulation import simBetaPoisson

In [ ]:
import pystan

In [ ]:
import pandas as pd

In [ ]:
UMI = pd.read_csv('data/nature_paper/SS3_c57_UMIs_concat.csv', index_col=0)
params = pd.read_pickle('data/nature_paper/SS3_c57_UMIs_concat_ML.pkl')

In [ ]:
mean_c57 = UMI.mean(axis=1)

In [ ]:
gof_df = pd.read_csv('good_fibroblast_genes.csv', index_col=0)
goodgenes = gof_df.index.values

In [ ]:
params = params[params[1]][0]
params = pd.DataFrame(params)

In [ ]:
sm = pystan.StanModel(file='allelic_imbalance.stan')

In [ ]:
def simAI(pars, n = 50, p = 1):
    a1 = np.random.binomial(simBetaPoisson(pars, size=n), p)
    a2 = np.random.binomial(simBetaPoisson(pars, size=n), p)
    dat = {'y':[np.sum(a1<a2), np.sum(a2<a1), np.sum(a1==a2)]}
    samp = sm.sampling(data=dat, iter=10000, chains=4)
    s = samp.summary()
    summary = pd.DataFrame(s['summary'], columns=s['summary_colnames'], index=s['summary_rownames'])
    if summary['mean']['allele1_normed'] > summary['mean']['allele2_normed']:
        return [summary['mean']['allele1_normed'], summary['2.5%']['allele1_normed'],summary['97.5%']['allele1_normed']]
    else:
        return [summary['mean']['allele2_normed'], summary['2.5%']['allele2_normed'],summary['97.5%']['allele2_normed']]

In [ ]:
sim_values = {}

for gene in goodgenes:
    for n_cells in [10,20,50,100,1000,10000]:
        print(n_cells)
        if n_cells in sim_values.keys():
            sim_values[n_cells] = np.append(sim_values[n_cells],simAI(params.loc[gene][0],n=n_cells))
        else:
            sim_values[n_cells] = np.array([simAI(params.loc[gene][0], n=n_cells)])

In [ ]:
for n in sim_df:
    cdf = sim_df[n].value_counts().sort_index().cumsum()
    cdf.plot(label=n)
plt.legend()
plt.xlabel('Allelic Imbalance')
plt.ylabel('Cumulative frequency')
plt.savefig('figures/Fig5B.svg')
plt.show()

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(7,7), sharex=True, sharey=True)
for n,ax in zip(sim_df, iterAX(axes)):
    ax.scatter(np.log10(mean_c57), sim_df[n], s=1)
plt.tight_layout()
plt.savefig('figures/Fig5C.svg')
plt.show()